In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import PolynomialFeatures


np.random.seed(2)

Read and display data:

In [ ]:
train_data = pd.read_csv('./bitrate_prediction/bitrate_train.csv')
test_data = pd.read_csv('./bitrate_prediction/bitrate_test.csv')

In [ ]:
report = ProfileReport(train_data)
#report.to_file('./reg_report.html')

In [ ]:
test_data.head()

There are no missing values. All the columns contain numerical data, no encoding is required.

In [ ]:
X_train, y_train = train_data.drop('target', axis=1), train_data['target']  
X_test, y_test = test_data.drop('target', axis=1), test_data['target']

## Visualization

Visualize data by projecting it to two dimensions with PCA

In [ ]:
from sklearn.decomposition import PCA

# scale features before PCA
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X_train)

model = PCA(0.95)
projected = model.fit_transform(scaled_data)

In [ ]:
plt.title('Explained variance ratio')
plt.bar(range(len(model.explained_variance_ratio_)), model.explained_variance_ratio_)
plt.xlabel('Component')
plt.ylabel('% Variance')

Projection to principal component vectors does not help to save as much information as possible in the first components. This could mean unapplicability of the method or influence of some noise in the data.

Interaction between first two component projections:

In [ ]:
plt.title('Projected data to first two PC')
plt.scatter(projected[:, 0], projected[:, 1])
plt.xlabel('First component')
plt.ylabel('Second component')

The scatter plot shows that some points in the projection are far from others which could imply presence of outliers.

In [ ]:
import seaborn as sns

sns.pairplot(train_data[['bitrate_mean', 'bitrate_std', 'fps_mean', 'fps_std', 'rtt_mean', 'target']])

## Feature Selection

Correlation between features:

In [ ]:
sns.heatmap(train_data.corr())

There are features which have low correlation with target that can be excluded. Some features are highly correlated with each other and introduce multicollinearity to the regression problem.

Dropped frame related features have 97% of constant zero values and are not correlated with target, therefore we can remove them from consideration. On the other hand, bitrate_mean and bitrate_std features show high correlation with target variable as they describe the same thing.

L1 regularized models give sparse coefficients which can be interpreted as their importance to the model. Unimportant features get 0 weight score. Feature selection with Lasso coefficients:

In [ ]:
feature_selection_pipe = Pipeline([
        ('preprocessing', StandardScaler()), 
        ('model', Lasso())
])
# train Linear Regression with l1 regularization
feature_selection_pipe.fit(X_train, y_train)

cols = X_train.columns
weights = feature_selection_pipe['model'].coef_

# visualize weights
plt.bar(range(len(weights)), weights)
plt.ylim(top=0.3*max(weights))
plt.title('Weights of Lasso model for each feature')
plt.ylabel('Weight')
plt.xlabel('Feature')
_ = plt.xticks(ticks=range(len(cols)), labels=cols, rotation=90)

In [ ]:
# select columns with non-zero weight score
cols_important = [col for i, col in enumerate(cols) if weights[i]]
print('Selected columns: ', cols_important)
cols_to_exclude = list(set(cols) - set(cols_important))
print('Excluded columns: ', cols_to_exclude)

The feature 'dropped_frames_max' can be dropped.

In [ ]:
X_train = X_train.drop(cols_to_exclude, axis=1)
X_test = X_test.drop(cols_to_exclude, axis=1)

Further feature selection can be performed

In [ ]:
# scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
from sklearn.feature_selection import mutual_info_regression

# calculate mutual inf scores
mi = mutual_info_regression(X_train_scaled, y_train)
mutual_info = pd.Series(mi, index=X_train.columns)

# visualize
mutual_info.sort_values(ascending=False).plot.bar(figsize=(10, 4))
plt.title('Mutual information of features with target')
plt.ylabel('Mutual information score')

Select features based on their mutual information score with target:

In [ ]:
from sklearn.feature_selection import SelectKBest

mi_selector = SelectKBest(mutual_info_regression, k=6)
mi_selector.fit(X_train_scaled, y_train)
mi_selected_feats = X_train.columns[mi_selector.get_support()]
print('Selected features based on mutual information: ', mi_selected_feats.tolist())

Feature importance to the model can be obtained by comparing scores from training with dataset including this feature or without it.

In [ ]:
from sklearn.feature_selection import RFECV, RFE

rfe = RFE(
    estimator=Lasso(),
    step=1,
    n_features_to_select=2
)

rfe.fit(X_train_scaled, y_train)

print('Ranked features: ')
print(pd.Series(X_train.columns[rfe.ranking_.argsort()]))

This process can be done iteratively with cross validation considering every combination of features and saving the best match.

In [ ]:
from sklearn.model_selection import StratifiedKFold

rfecv = RFECV(
    estimator=Lasso(),
    step=2,
    cv=5,
    min_features_to_select=2
)
rfecv.fit(X_train_scaled, y_train)
print('Selected features: ', X_train.columns[rfecv.support_].tolist())
print('Dropped features: ', set(X_train.columns) - set(X_train.columns[rfecv.support_]))

All methods agree that dropped_frames features can be excluded without major performance drop. Besides, these columns are mostly zeros. Deleting features helps to

In [ ]:
selected_cols = X_train.columns[rfecv.support_].tolist()
print('Final selected columns: ', selected_cols)

## Model

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def rmse(y_true, y_pred):
    """
    Root of Mean Squared Error
    :param y_true: ground truth values
    :param y_pred: predicted values
    :return: rmse score
    """
    return np.sqrt(mean_squared_error(y_true, y_pred))

def get_regr_metrics(y_true, y_pred):
    """
    Calculation of regression metrics
    :param y_true: ground truth values
    :param y_pred: predicted values
    :return: pd.Series of metric values
    """
    metrics = {'MSE': mean_squared_error(y_true, y_pred), 
              'RMSE':rmse(y_true, y_pred),
              'MAE':mean_absolute_error(y_true, y_pred),
               'R2':r2_score(y_true, y_pred)}
    
    return pd.Series(metrics)

In [ ]:
def fit_predict(model, X_train, y_train, X_test):
    """
    Function to perform train and predict
    :param model: model to use
    :param X_train: train data
    :param y_train: target
    :param X_test: test data
    :return: predicted values for train and test data
    """
    y_pred_train = model.fit_predict(X_train, y_train)
    y_pred_test = model.predict(X_test)    
    return y_pred_train, y_pred_test

def print_metrics(y_train, y_pred_train, y_test, y_pred_test):
    """
    Function to get train and test metrics
    :param y_train: ground truth for train data
    :param y_pred_train: predicted values for train data
    :param y_test: ground truth for test data
    :param y_pred_test: predicted values for test data
    :return: DataFrame with train and test metrics
    """
    train_metrics = get_regr_metrics(y_train, y_pred_train)
    test_metrics = get_regr_metrics(y_test, y_pred_test)
    return pd.concat([train_metrics, test_metrics], axis=1, keys=['train', 'test']).round(3)

Baseline is training linear regression over all initial features:

In [ ]:
linear_regr = Pipeline([
    ('preprocessing', StandardScaler()), 
    ('model', LinearRegression())
])

linear_regr.fit(X_train, y_train)
y_train_pred = linear_regr.predict(X_train)
y_test_pred = linear_regr.predict(X_test)

print_metrics(y_train, y_train_pred, y_test, y_test_pred) 

Traininng linear model on selected features:

In [ ]:
linear_regr.fit(X_train[selected_cols], y_train)
y_train_pred = linear_regr.predict(X_train[selected_cols])
y_test_pred = linear_regr.predict(X_test[selected_cols])

print_metrics(y_train, y_train_pred, y_test, y_test_pred) 

Performance did not drop much, but there are less number of predictors

 - Models with regularization

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

regr_lasso = Pipeline([
    ('preprocessing', StandardScaler()), 
    ('model', Lasso())
])

search_tool = RandomizedSearchCV(regr_lasso, {'model__alpha':uniform(loc=0, scale=1)}, 
                                 scoring='r2')
search_tool.fit(X_train[selected_cols], y_train)
print('Best parameter value alpha = ', search_tool.best_params_['model__alpha'], '\n')
print('Best score: ', search_tool.best_score_)

regr_lasso['model'].alpha = search_tool.best_params_['model__alpha']

regr_lasso.fit(X_train[selected_cols], y_train)
y_train_pred = regr_lasso.predict(X_train[selected_cols])
y_test_pred = regr_lasso.predict(X_test[selected_cols])

print_metrics(y_train, y_train_pred, y_test, y_test_pred) 

In [ ]:
regr_ridge = Pipeline([
    ('preprocessing', StandardScaler()), 
    ('model', Ridge())
])

search_tool = RandomizedSearchCV(regr_ridge, {'model__alpha':uniform(loc=0, scale=1)}, 
                                 scoring='r2')
search_tool.fit(X_train[selected_cols], y_train)
print('Best parameter value alpha = ', search_tool.best_params_['model__alpha'], '\n')
print('Best score: ', search_tool.best_score_)

regr_ridge['model'].alpha = search_tool.best_params_['model__alpha']

regr_ridge.fit(X_train[selected_cols], y_train)
y_train_pred = regr_ridge.predict(X_train[selected_cols])
y_test_pred = regr_ridge.predict(X_test[selected_cols])

print_metrics(y_train, y_train_pred, y_test, y_test_pred) 

- Polynomial Features

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

reg_poly_ridge = Pipeline([
    ('poly', PolynomialFeatures()), 
    ('preprocessing', StandardScaler()), 
    ('model', Ridge())
])

search_tool = RandomizedSearchCV(reg_poly_ridge, {'model__alpha':uniform(loc=0, scale=1),
                                                      'poly__degree':[1, 2, 3]}, 
                                 scoring='r2')
search_tool.fit(X_train[selected_cols], y_train)
print('Best parameter value alpha = ', search_tool.best_params_['model__alpha'], '\n')
print('Degree: ', search_tool.best_params_['poly__degree'], '\n')
print('Best score: ', search_tool.best_score_)

reg_poly_ridge['model'].alpha = search_tool.best_params_['model__alpha']
reg_poly_ridge['poly'].degree = search_tool.best_params_['poly__degree']

reg_poly_ridge.fit(X_train[selected_cols], y_train)
y_train_pred = reg_poly_ridge.predict(X_train[selected_cols])

y_test_pred = reg_poly_ridge.predict(X_test[selected_cols])

print_metrics(y_train, y_train_pred, y_test, y_test_pred) 

In [ ]:
interaction_regr = Pipeline([
    ('interaction', PolynomialFeatures(degree=2, interaction_only=True)),
    ('preprocessing', StandardScaler()), 
    ('model', Lasso(0.001))
])

interaction_regr.fit(X_train[selected_cols], y_train)
y_train_pred = interaction_regr.predict(X_train[selected_cols])
y_test_pred = interaction_regr.predict(X_test[selected_cols])

print_metrics(y_train, y_train_pred, y_test, y_test_pred) 

### Outlier detection

Linear models are highly affected by the outliers. As there is no assumption of normal distribution in given data, interquartile range method can be used to detect them.

In [ ]:
iqr = X_train[selected_cols].quantile(0.75) - X_train[selected_cols].quantile(0.25)
up_bound = X_train[selected_cols].quantile(0.75) + iqr * 1.5
low_bound = X_train[selected_cols].quantile(0.25) - iqr * 1.5
outliers_mask = ((X_train[selected_cols] > up_bound) | (X_train[selected_cols] < low_bound)).sum(axis=1) > 0
X_train_clean = X_train[~outliers_mask][selected_cols].copy()
y_train_clean = y_train[~outliers_mask].copy()

print('Percentage of outliers: ', 1 - X_train_clean.shape[0] / X_train.shape[0])

Data distribution and paired scatter plots after removing outliers:

In [ ]:
sns.pairplot(X_train_clean.assign(target=y_train_clean))

There is no linear correlation in the new features.

In [ ]:
reg_ridge = Pipeline([
    ('preprocessing', StandardScaler()), 
    ('model', Ridge())
])

search_tool = RandomizedSearchCV(reg_ridge, {'model__alpha':uniform(loc=0, scale=1)}, 
                                 scoring='r2')
search_tool.fit(X_train_clean, y_train_clean)
print('Best parameter value alpha = ', search_tool.best_params_['model__alpha'], '\n')
print('Best score: ', search_tool.best_score_)
reg_poly_ridge['model'].alpha = search_tool.best_params_['model__alpha']

reg_poly_ridge.fit(X_train_clean, y_train_clean)
y_train_pred = reg_poly_ridge.predict(X_train_clean)

y_test_pred = reg_poly_ridge.predict(X_test[selected_cols])

print_metrics(y_train_clean, y_train_pred, y_test, y_test_pred) 

Removing outliers hurt performance of a model, indicating that useful information was deleted.

As another approach, density based methods can be used:

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor()
yhat = lof.fit_predict(X_train[selected_cols])
outliers_mask = yhat == -1

vals, counts = np.unique(yhat, return_counts=True)
print('Percentage of outliers: ', counts[0] / train_data.shape[0])

In [ ]:
X_train_clean = X_train[selected_cols][~outliers_mask]
y_train_clean = y_train[~outliers_mask]

In [ ]:
sns.pairplot(X_train_clean.assign(target=y_train_clean))

In [ ]:
search_tool.fit(X_train_clean, y_train_clean)
print('Best parameter value alpha = ', search_tool.best_params_['model__alpha'], '\n')
print('Degree: ', search_tool.best_params_['poly__degree'], '\n')
reg_poly_ridge['model'].alpha = search_tool.best_params_['model__alpha']
reg_poly_ridge['poly'].degree = search_tool.best_params_['poly__degree']

reg_poly_ridge.fit(X_train_clean, y_train_clean)
y_train_pred = reg_poly_ridge.predict(X_train_clean)

y_test_pred = reg_poly_ridge.predict(X_test[selected_cols])

print_metrics(y_train_clean, y_train_pred, y_test, y_test_pred) 

Outlier detection hurts the performance of model. Perhaps there are different methods or the distribution is just too wide.